# Dog Breed classification

#### Libraries

In [40]:
# plot
import matplotlib.pyplot as plt

# tools
import os
from glob import glob
from shutil import copyfile            
from tqdm import tqdm
import random
import time

# numpy
import numpy as np

# sklearn
from sklearn import metrics
from sklearn.datasets import load_files   

# keras
import keras
from keras.utils import np_utils

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img,image
from keras import optimizers

from keras.applications import vgg16
from keras.applications.vgg16 import VGG16
from keras.applications.resnet import ResNet50
from keras.applications import ResNet50
from keras.applications import resnet50

from keras.layers import Input, Dense, Flatten, GlobalAveragePooling2D, Dropout
from keras.models import Model, Sequential

from keras.callbacks import ModelCheckpoint, History, EarlyStopping

#### Constants

In [36]:
BATCH_SIZE = 16
EPOCHS = 10
NUM_CLASSES = 120
INPUT_SHAPE = (256, 256, 3)
IMAGE_SIZE = (256, 256)
SEED = 42

DATA_PATH = '../data/'

USE_DROPOUT = True
DROPOUT_RATE = 0.5


## I. Datas

### I.1. Split train test

In [4]:
def img_train_test_split(img_source_dir, train_size, validation_size):
    """
    Randomly splits images over a train and validation folder, while preserving the folder structure
    
    Parameters
    ----------
    * img_source_dir : string
        Path to the folder with the images to be split. Can be absolute or relative path   
        
    * train_size : float
        Proportion of the original images that need to be copied in the subdirectory in the train folder
    """    
    if not (isinstance(img_source_dir, str)):
        raise AttributeError('img_source_dir must be a string')
        
    if not os.path.exists(img_source_dir):
        raise OSError('img_source_dir does not exist')
        
    if not (isinstance(train_size, float)):
        raise AttributeError('train_size must be a float')
        
    # Set up empty folder structure if not exists
    if not os.path.exists('../data'):
        os.makedirs('../data')
    else:
        if not os.path.exists('../data/train'):
            os.makedirs('../data/train')
        if not os.path.exists('../data/validation'):
            os.makedirs('../data/validation')
        if not os.path.exists('../data/test'):
            os.makedirs('../data/test')
            
    # Get the subdirectories in the main image folder
    subdirs = [subdir for subdir in os.listdir(img_source_dir) if os.path.isdir(os.path.join(img_source_dir, subdir))]

    for subdir in subdirs:
        subdir_fullpath = os.path.join(img_source_dir, subdir)
        if len(os.listdir(subdir_fullpath)) == 0:
            print(subdir_fullpath + ' is empty')
            break

        train_subdir = os.path.join('../data/train', subdir)
        validation_subdir = os.path.join('../data/validation', subdir)
        test_subdir = os.path.join('../data/test', subdir)

        # Create subdirectories in train and validation folders
        if not os.path.exists(train_subdir):
            os.makedirs(train_subdir)

        if not os.path.exists(validation_subdir):
            os.makedirs(validation_subdir)
            
        if not os.path.exists(test_subdir):
            os.makedirs(test_subdir)

        train_counter = 0
        validation_counter = 0
        test_counter = 0

        # Randomly assign an image to train or validation folder
        for filename in os.listdir(subdir_fullpath):
            if filename.endswith(".jpg") or filename.endswith(".png"): 
                fileparts = filename.split('.')

                if random.uniform(0, 1) <= train_size:
                    copyfile(os.path.join(subdir_fullpath, filename),
                             os.path.join(train_subdir, str(train_counter) + '.' + fileparts[1]))
                    train_counter += 1
                elif random.uniform(0, 1) <= validation_size:
                    copyfile(os.path.join(subdir_fullpath, filename),
                             os.path.join(validation_subdir, str(validation_counter) + '.' + fileparts[1]))
                    validation_counter += 1
                else :
                    copyfile(os.path.join(subdir_fullpath, filename),
                             os.path.join(test_subdir, str(test_counter) + '.' + fileparts[1]))
                    test_counter += 1

In [ ]:
img_train_test_split('../data/Images', 0.7, 0.4)

### I.2. Load datasets

In [55]:
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 120)
    return dog_files, dog_targets

In [58]:
train_img, train_targets = load_dataset('../data/train')
val_img, val_targets = load_dataset('../data/validation')
test_img, test_targets = load_dataset('../data/test')

### I.3. Tensors 

In [53]:
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [ ]:
vgg16_train_tensors = paths_to_tensor(train_img).astype('float32')/255
vgg16_val_tensors = paths_to_tensor(val_img).astype('float32')/255
vgg16_test_tensors = paths_to_tensor(test_img).astype('float32')/255

100%|██████████| 14443/14443 [00:48<00:00, 300.22it/s]


### I.4. Data augmentation

In [17]:
def get_generators(function):
    '''
    Give us the generators for the train, val and test set.
    
    Parameters
    ----------
    function : preprocessing function use for data augmentation
    '''
    
    generators = []
    
    # augmentation configuration for training
    train_datagen = ImageDataGenerator(rotation_range=20,
                                       zoom_range=0.2,
                                       horizontal_flip=True,
                                       fill_mode='nearest',
                                       preprocessing_function=function.preprocess_input)
    
    # generators for train
    train_gen = train_datagen.flow_from_directory('../data/train/',
                                                  target_size = IMAGE_SIZE,
                                                  batch_size = BATCH_SIZE,
                                                  shuffle=True,
                                                  class_mode = 'categorical',
                                                  seed=SEED)
    #generators for val
    val_gen = train_datagen.flow_from_directory('../data/validation/',
                                                target_size = IMAGE_SIZE,
                                                batch_size = BATCH_SIZE,
                                                class_mode = 'categorical',
                                                seed=SEED)
    # rescaling
    test_datagen = ImageDataGenerator(preprocessing_function=function.preprocess_input)
    
    #generators for test
    test_gen = train_datagen.flow_from_directory('../data/test/',
                                                 target_size = IMAGE_SIZE,
                                                 batch_size = BATCH_SIZE,
                                                 class_mode = 'categorical',
                                                 seed = SEED)
    
    # list of generators
    generators.append(train_gen)
    generators.append(val_gen)
    generators.append(test_gen)
    
    return generators

Vgg16 generators

In [18]:
vgg16_train_gen, vgg16_val_gen, vgg16_test_gen = get_generators(vgg16)

Found 14443 images belonging to 120 classes.
Found 2417 images belonging to 120 classes.
Found 3720 images belonging to 120 classes.


ResNet generators

In [22]:
resnet_train_gen, resnet_val_gen, resnet_test_gen = get_generators(resnet50)

Found 14443 images belonging to 120 classes.
Found 2417 images belonging to 120 classes.
Found 3720 images belonging to 120 classes.


## II. Models

### II.1. Compile

In [7]:
def model(model):
    """
    Compile a cnn model
    
    """
    pretrained_model= model(include_top=False, weights="imagenet", input_shape=INPUT_SHAPE)
    
    for layer in pretrained_model.layers:
        layer.trainable = False
        
    x = pretrained_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation="relu")(x)
    x = Dropout(DROPOUT_RATE)(x)
    predictions = Dense(120, activation='softmax')(x)

    model = Model(inputs = pretrained_model.input, outputs=predictions)

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

Vgg16

In [25]:
vgg16_model = model(VGG16)

Resnet50 

In [27]:
resnet_model = model(ResNet50)

/home/bayet/.virtualenvs/py3cv4/lib/python3.5/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


### II.2. Train model

#### Callbacks 

In [32]:
class TimeHistory(keras.callbacks.Callback):

    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time()-self.epoch_time_start)

In [33]:
def get_callbacks(model_name):
    save_callback = ModelCheckpoint(DATA_PATH + 'models/%s.{epoch:02d}-{val_acc:.2f}.hdf5' %model_name, 
                                    monitor='val_acc',
                                    verbose=1, 
                                    save_best_only=True,
                                    save_weights_only=False, 
                                    mode='auto',
                                    period=2)
    time_callback = TimeHistory()
    early_stop_callback = EarlyStopping(monitor='val_acc', patience=2)
    
    return [save_callback, time_callback, early_stop_callback]

Vgg16 callbacks 

In [38]:
vgg16_callbacks = get_callbacks(vgg16)

Resnet callbacks 

In [44]:
resnet_callbacks = get_callbacks(resnet50)

#### Train model

In [34]:
def train_model(model, callbacks, generator, val_generator):
    
    history = model.fit_generator(generator=generator,
                                  validation_data=val_generator,
                                  epochs=EPOCHS,
                                  steps_per_epoch= len(generator.classes) // BATCH_SIZE,
                                  validation_steps= len(generator.classes) // BATCH_SIZE,
                                  workers=4,
                                  callbacks = callbacks)
    
    return model, history

In [41]:
vgg16 = train_model(vgg16_model, vgg16_callbacks, vgg16_train_gen, vgg16_val_gen )

Epoch 1/10
 10/902 [..............................] - ETA: 59:59 - loss: 13.8371 - accuracy: 0.0188

KeyboardInterrupt: 

In [45]:
resnet = train_model(resnet_model, resnet_callbacks, resnet_train_gen, resnet_val_gen )

Epoch 1/10
  2/902 [..............................] - ETA: 40:56 - loss: 4.9005 - accuracy: 0.0625

KeyboardInterrupt: 

# III. Evaluation of models 

### II.1. Vizualization

In [51]:
def plot_acc(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    t = np.linspace(1, len(acc), len(acc)).flatten()
    plt.plot(t, acc, 'navy', label='train accuracy')
    plt.plot(t, val_acc, 'lightblue', label='validation accuracy')

    plt.grid(True)
    plt.xlabel('Epochs')
    plt.title('Validation accuracy')
    plt.legend(loc=2)
    plt.show();

SyntaxError: invalid syntax (<ipython-input-51-da53cc4480d4>, line 1)

In [50]:
t = np.linspace(1, 5, 10).flatten()
t

array([1.        , 1.44444444, 1.88888889, 2.33333333, 2.77777778,
       3.22222222, 3.66666667, 4.11111111, 4.55555556, 5.        ])

In [ ]:
def final_predict(img_path, topk = 3):
    
    # obtain predicted vector
    
    predicted_vector = model.predict(preprocess_input(path_to_tensor(img_path)))
    
    # return dog breed that is predicted by the model
    # print(dog_names[np.argmax(predicted_vector)])
    results = sorted(enumerate(predicted_vector[0]), reverse=True, key=lambda x:x[1])
    classes_ind = [x[0] for x in results]
    classes = [dog_names[x] for x in classes_ind][:topk]
    probs = [x[1] for x in results][:topk]
    
    # plot results
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))
    
    img = mpimg.imread(img_path)
    ax1.imshow(img)
    
    ind = np.arange(len(classes))
    ax2.bar(ind, probs, align='center', alpha = .75)
    ax2.set_xticks(ind)
    ax2.set_xticklabels(classes, rotation=90)